In [6]:
import numpy as np
import cv2

def initialize_centroids(data, k):
    """Randomly initialize centroids from the data points."""
    if data.shape[0] < k:
        raise ValueError("The number of clusters cannot exceed the number of data points.")
    indices = np.random.choice(data.shape[0], k, replace=False)
    return data[indices]

def assign_clusters(data, centroids):
    """Assign data points to the nearest centroid."""
    distances = np.linalg.norm(data - centroids[:, np.newaxis], axis=2)
    return np.argmin(distances, axis=0)

def update_centroids(data, labels, k):
    """Update centroid positions as the mean of their respective data points."""
    new_centroids = np.zeros((k, data.shape[1]))
    for i in range(k):
        cluster_points = data[labels == i]
        if cluster_points.size == 0:
            # Avoid empty clusters by reinitializing to a random data point
            new_centroids[i] = data[np.random.choice(data.shape[0])]
        else:
            new_centroids[i] = cluster_points.mean(axis=0)
    return new_centroids

def kmeans(data, k, max_iters=100):
    """The main k-means clustering algorithm."""
    centroids = initialize_centroids(data, k)
    for _ in range(max_iters):
        labels = assign_clusters(data, centroids)
        new_centroids = update_centroids(data, labels, k)
        if np.allclose(centroids, new_centroids):
            break
        centroids = new_centroids
    return centroids, labels

def compute_likelihood(pixel, cluster_centers):
    """Compute the likelihood of a pixel belonging to clusters represented by the cluster centers."""
    w_k = 0.1
    distances = np.linalg.norm(cluster_centers - pixel, axis=1)
    return np.sum(w_k * np.exp(-distances))

# Load images using OpenCV
original_image = cv2.imread('van-Gogh.png')
aux_image = cv2.imread('van-Gogh-stroke.png')

# Apply Gaussian blur for noise reduction
blurred_original_image = cv2.GaussianBlur(original_image, (5, 5), 0)
blurred_aux_image = cv2.GaussianBlur(aux_image, (5, 5), 0)

# Convert auxiliary image to HSV color space for more reliable color detection
aux_hsv = cv2.cvtColor(blurred_aux_image, cv2.COLOR_BGR2HSV)

# Define color thresholds for red and blue in HSV space
red_lower = np.array([0, 120, 70])
red_upper = np.array([10, 255, 255])
blue_lower = np.array([110, 150, 50])
blue_upper = np.array([130, 255, 255])

# Create masks for red and blue
foreground_mask = cv2.inRange(aux_hsv, red_lower, red_upper)
background_mask = cv2.inRange(aux_hsv, blue_lower, blue_upper)

# Use masks to extract seed pixels from the blurred original image
foreground_pixels = blurred_original_image[foreground_mask == 255].reshape(-1, 3)
background_pixels = blurred_original_image[background_mask == 255].reshape(-1, 3)

# Perform K-Means clustering on the foreground and background seed pixels
n_clusters = 64
centroids_fg, _ = kmeans(foreground_pixels, n_clusters)
centroids_bg, _ = kmeans(background_pixels, n_clusters)

height, width, _ = original_image.shape
segmentation_mask = np.zeros((height, width), dtype=np.uint8)

# Classify each pixel in the original image based on the computed likelihoods
for i in range(height):
    for j in range(width):
        pixel = blurred_original_image[i, j]
        likelihood_fg = compute_likelihood(pixel, centroids_fg)
        likelihood_bg = compute_likelihood(pixel, centroids_bg)
        segmentation_mask[i, j] = 1 if likelihood_fg > likelihood_bg else 0

# Post-process the segmentation mask to remove noise
kernel = np.ones((5, 5), np.uint8)
segmentation_mask = cv2.morphologyEx(segmentation_mask.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
segmentation_mask = cv2.morphologyEx(segmentation_mask, cv2.MORPH_OPEN, kernel)

# Conduct connected component analysis to remove small noisy regions
num_labels, labels_im = cv2.connectedComponents(segmentation_mask)
for label in range(1, num_labels):
    if np.sum(labels_im == label) < 500:  # Threshold for noise components
        segmentation_mask[labels_im == label] = 0

# Fill in gaps using contour analysis
contours, _ = cv2.findContours(segmentation_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for cnt in contours:
    cv2.drawContours(segmentation_mask, [cnt], 0, 1, -1)

# Apply the segmentation mask to the original image
foreground_color = cv2.bitwise_and(original_image, original_image, mask=segmentation_mask)

# Convert the original image to grayscale and then to 3 channels
background_grayscale = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
background_color = cv2.cvtColor(background_grayscale, cv2.COLOR_GRAY2BGR)

# Combine color foreground with grayscale background
combined_image = cv2.add(foreground_color, cv2.bitwise_and(background_color, background_color, mask=~segmentation_mask))

# Save the colored foreground with desaturated background image
output_colored_path = '2-van-segmentation.png'
cv2.imwrite(output_colored_path, combined_image)

print(f"Segmentation with colored foreground saved to {output_colored_path}")


Segmentation with colored foreground saved to 3-2-Lady-segmentation.png
